# Import required modules

In [291]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn import cross_validation
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns
import glob
import os
from sklearn.cross_validation import cross_val_score
plt.style.use('ggplot')

# Define Functions

In [292]:
#A function to calculate the results
def result(x):
	conditions = [
	(x['GoalsFor'] > x['GoalsAgainst']),
	(x['GoalsFor'] == x['GoalsAgainst']),
	(x['GoalsFor'] < x['GoalsAgainst'])
	]
	choices = [1,0,-1]
	x['Result'] = np.select(conditions, choices)
	return x

In [293]:
#A function to calculate the results
def HomeResult(x):
	conditions = [
	(x['HomeGoals'] > x['AwayGoals']),
	(x['HomeGoals'] == x['AwayGoals']),
	(x['HomeGoals'] < x['AwayGoals'])
	]
	choices = [1,0,-1]
	x['HomeResult'] = np.select(conditions, choices)
	return x

def AwayResult(x):
	conditions = [
	(x['AwayGoals'] > x['HomeGoals']),
	(x['AwayGoals'] == x['HomeGoals']),
	(x['AwayGoals'] < x['HomeGoals'])
	]
	choices = [1,0,-1]
	x['AwayResult'] = np.select(conditions, choices)
	return x

In [294]:
def joined_result(x):
	conditions = [
	(x['Result'] == 'H'),
	(x['Result'] == 'D'),
	(x['Result'] == 'A')
	]
	choices = [1,0,-1]
	x['ResultValue'] = np.select(conditions, choices)
	return x

# Data Preperation

Read CSV Data

In [295]:
#full_stats=pd.read_csv(r'C:\Users\dbrown\Documents\Misc\Football\16_17\overall.csv')
full_stats = pd.DataFrame()
for year in ['1011','1112','1213','1314','1415','1516','1617','1718']:
    for div in ['E0','E1','E2','E3','SC0','SC1','SC2','SC3','B1','F1','F2','SP1','SP2','I1','I2','D1','D2','N1','P1','G1','T1']:
        url = 'http://www.football-data.co.uk/mmz4281/%s/%s.csv' % (year,div)
        #results = results.append(pd.read_csv(url,usecols=['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR']))
        full_stats = full_stats.append(pd.read_csv(url,usecols=['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR']))
    
fixtures_url="http://www.football-data.co.uk/fixtures.csv"
fixtures=pd.read_csv(fixtures_url,usecols=['Date','Div','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])

Define base dataframes

In [296]:
full_results_combo = full_stats.append(fixtures)
#full_results_combo2.fillna(0,inplace=True)
full_results_combo.drop_duplicates(subset=['Date','Div','HomeTeam','AwayTeam'],keep='last',inplace=True)
full_results_combo.set_index(['Date','Div','HomeTeam','AwayTeam'],inplace=True)
full_results_combo.rename(columns={'FTHG':'HomeGoals','FTAG':'AwayGoals','FTR':'Result'},inplace='True')
full_results_combo = joined_result(full_results_combo2)
full_results_combo = HomeResult(full_results_combo2)
full_results_combo = AwayResult(full_results_combo2)
full_results_combo['HomeGoalDiff'] = full_results_combo2['HomeGoals'] - full_results_combo2['AwayGoals']
full_results_combo['AwayGoalDiff'] = full_results_combo2['AwayGoals'] - full_results_combo2['HomeGoals']
full_results_combo['H2H'] = full_results_combo2['ResultValue'].groupby(level=['HomeTeam','AwayTeam']).apply(lambda x: pd.rolling_mean(x.shift(),10000,min_periods=3))

Set 2-6 game form values

In [297]:
for num in range(2,7):
        home = 'Home'
        away = 'Away'
        HomeGameForm="%s%sGameForm" % (home,num)
        HomeGameGoalsFor="%s%sGameGoalsFor" % (home,num)
        HomeGameGoalDiff="%s%sGameGoalDiff" % (home,num)
        AwayGameForm="%s%sGameForm" % (away,num)
        AwayGameGoalsFor="%s%sGameGoalsFor" % (away,num)
        AwayGameGoalDiff="%s%sGameGoalDiff" % (away,num)
        full_results_combo[HomeGameForm] = full_results_combo['HomeResult'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        full_results_combo[HomeGameGoalsFor] = full_results_combo['HomeGoals'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        full_results_combo[HomeGameGoalDiff] = full_results_combo['HomeGoalDiff'].groupby(level='HomeTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        full_results_combo[AwayGameForm] = full_results_combo['AwayResult'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        full_results_combo[AwayGameGoalsFor] = full_results_combo['AwayGoals'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))
        full_results_combo[AwayGameGoalDiff] = full_results_combo['AwayGoalDiff'].groupby(level='AwayTeam').apply(lambda x: pd.rolling_mean(x.shift(),num))

Set 2-6 game form difference values

In [298]:
for num in range(2,7):
        home = 'Home'
        away = 'Away'
        HomeGameForm="%s%sGameForm" % (home,num)
        HomeGameGoalsFor="%s%sGameGoalsFor" % (home,num)
        HomeGameGoalDiff="%s%sGameGoalDiff" % (home,num)
        AwayGameForm="%s%sGameForm" % (away,num)
        AwayGameGoalsFor="%s%sGameGoalsFor" % (away,num)
        AwayGameGoalDiff="%s%sGameGoalDiff" % (away,num)
        GameFormDiff = "%sGameFormDiff" % (num)
        GameGoalsForDiff = "%sGameGoalsForDiff" % (num)
        GameGoalDiffDiff = "%sGameGoalDiffDiff" % (num)
        full_results_combo[GameFormDiff] = full_results_combo[HomeGameForm] - full_results_combo[AwayGameForm]
        full_results_combo[GameGoalsForDiff] = full_results_combo[HomeGameGoalsFor] - full_results_combo[AwayGameGoalsFor]
        full_results_combo[GameGoalDiffDiff] = full_results_combo[HomeGameGoalDiff] - full_results_combo[AwayGameGoalDiff]

Deal with NaN values

In [299]:
full_results_combo2_zeroed = full_results_combo2.fillna(value = {'HomeGoals': 0,'AwayGoals': 0,'Result': 0,'ResultValue': 0,'HomeResult': 0,'AwayResult': 0,'HomeGoalDiff': 0,'AwayGoalDiff': 0})
full_results_combo2_zeroed_dropped = full_results_combo2_zeroed.dropna()
full_results_combo2_dropped = full_results_combo2.dropna()
full_results_combo2_zeroed_dropped_reset = full_results_combo2_zeroed_dropped.reset_index()

In [311]:
stats_latest = full_results_combo2_zeroed_dropped_reset.groupby('HomeTeam').last()
stats_latest.reset_index(inplace=True)
stats_latest.set_index(['Date','Div','HomeTeam','AwayTeam'],inplace=True)
latest_fixtures = stats_latest[stats_latest["Result"]==0]

Get the latest stats

# Logistic Regression on HomeResult

In [302]:
#y = home_away_merge_idx_dropped['HomeResult']
#X = home_away_merge_idx_dropped[[
#    '3GameFormDiff', '3GameGoalsForDiff', '3GameGoalDiffDiff']]
#logreg = LogisticRegression()

In [303]:
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.8)
#logreg.fit(X_train, y_train)
#logreg.score(X_train, y_train)
#cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean()

In [327]:
columns = ['H2H',
       'Home2GameForm', 'Home2GameGoalsFor', 'Home2GameGoalDiff',
       'Home3GameForm', 'Home3GameGoalsFor', 'Home3GameGoalDiff',
       'Home4GameForm', 'Home4GameGoalsFor', 'Home4GameGoalDiff',
       'Home5GameForm', 'Home5GameGoalsFor', 'Home5GameGoalDiff',
       'Home6GameForm', 'Home6GameGoalsFor', 'Home6GameGoalDiff',
       'Away2GameForm', 'Away2GameGoalsFor', 'Away2GameGoalDiff',
       'Away3GameForm', 'Away3GameGoalsFor', 'Away3GameGoalDiff',
       'Away4GameForm', 'Away4GameGoalsFor', 'Away4GameGoalDiff',
       'Away5GameForm', 'Away5GameGoalsFor', 'Away5GameGoalDiff',
       'Away6GameForm', 'Away6GameGoalsFor', 'Away6GameGoalDiff',
       '2GameFormDiff', '2GameGoalsForDiff', '2GameGoalDiffDiff',
       '3GameFormDiff', '3GameGoalsForDiff', '3GameGoalDiffDiff',
       '4GameFormDiff', '4GameGoalsForDiff', '4GameGoalDiffDiff',
       '5GameFormDiff', '5GameGoalsForDiff', '5GameGoalDiffDiff',
       '6GameFormDiff', '6GameGoalsForDiff', '6GameGoalDiffDiff'
        ]

In [328]:
y = full_results_combo2_dropped['HomeResult']
X = full_results_combo2_dropped[columns]

logreg = LogisticRegression()

logreg.fit(X, y)
logreg.score(X, y)

y_pred = logreg.predict(X)

In [329]:
latest_fixtures_cut = latest_fixtures[columns]
probs = logreg.predict_proba(latest_fixtures_cut)
probs_df = pd.DataFrame(probs)
probs_df.rename(columns={0:'AwayWinProb',1:'DrawProb',2:'HomeWinProb'},inplace='True')
latest_fixtures_cut_noind = latest_fixtures_cut.reset_index()
pred_join = latest_fixtures_cut_noind.join(probs_df)

In [330]:
pred_join_cut = pred_join[["Date","Div","HomeTeam","AwayTeam","HomeWinProb"]]
pred_join_cut[pred_join_cut["HomeWinProb"]>0.65]

,Date,Div,HomeTeam,AwayTeam,HomeWinProb
42,23/02/18,T1,Galatasaray,Bursaspor,0.689248
76,25/02/18,F1,Paris SG,Marseille,0.713578


In [332]:
pred_join_cut_away = pred_join[["Date","Div","HomeTeam","AwayTeam","AwayWinProb"]]
pred_join_cut_away[pred_join_cut_away["AwayWinProb"]>0.55]

,Date,Div,HomeTeam,AwayTeam,AwayWinProb
29,24/02/18,SC3,Cowdenbeath,Peterhead,0.569638
